<a href="https://colab.research.google.com/github/avnishcodes/DiabetesPredictionSVM/blob/main/DiabetesPredictionSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/avnishcodes/DiabetesPredictionSVM/main/diabetes_prediction_dataset.csv"
df = pd.read_csv(url)

# Display first 5 rows
df.head()


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [2]:
#Check dataset info
df.info()
#check for missing values
print("\nMissing values:\n", df.isnull().sum())
#check unique values in target column
print("\nTarget  column values:\n", df['diabetes'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB

Missing values:
 gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

Target  col

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import uniform

# Use a smaller sample for tuning
X_sample = X_train[:5000]
y_sample = y_train[:5000]

# Define parameter range
param_dist = {
    'C': uniform(0.1, 10),
    'gamma': uniform(0.01, 1),
    'kernel': ['rbf', 'linear']
}

# Randomized search
search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, verbose=1, random_state=42)
search.fit(X_sample, y_sample)

# Best model
model = search.best_estimator_
print("Best Parameters:", search.best_params_)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))

# Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print("Specificity:", tn / (tn + fp))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'C': np.float64(7.896910002727692), 'gamma': np.float64(0.606850157946487), 'kernel': 'linear'}
Accuracy: 0.9613
Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98     18300
           1       0.91      0.60      0.73      1700

    accuracy                           0.96     20000
   macro avg       0.94      0.80      0.85     20000
weighted avg       0.96      0.96      0.96     20000

Specificity: 0.9946994535519126
